In [49]:
import mistletoe
import yaml
import re
import os
import glob
import pylatex
from mistletoe.base_renderer import BaseRenderer
from mistletoe.latex_renderer import LaTeXRenderer
import random
from crypt import vigenere_encrypt

In [50]:
from mistletoe import block_token, span_token, Document
from itertools import chain
import pylatex

class Error(Exception):
    """Base class for exceptions in this module."""
    pass

class QuestionsEnvironment(pylatex.base_classes.Environment):
    """
    A class representing a custom LaTeX environment.

    This class represents a custom LaTeX environment named
    ``questions``.
    """

    _latex_name = 'questions'
    packages = []

# Define a custom renderer back to markdown so that it can be further processed 
# into latex later
class QuestionMarker(span_token.SpanToken):
    pattern = re.compile(r"\[([ |x])\]\s+(.*)")    

    def __init__(self, match):
        self.marker = match.group(1)
        self.question = match.group(2)
        
class QuestionList(block_token.List):
    pattern = re.compile(r'(?:\d{0,9}[.)]|[+\-*]) {0,1}\[[ |x]\](?:[ \t]*$|[ \t]+)')

class QuestionRenderer(LaTeXRenderer):   
    def __init__(self, *extras, **kwargs):
        """
        Args:
            extras (list): allows subclasses to add even more custom tokens.
        KeywordArgs:
            language: document language (to be used with polyglossia)
            student_no: matriculation number of the student
            student_name: name of the student
            exam: exam name
            date: exam date
        """
        self.record_answers = False
        self.answers = []
        # TODO: check parameter coherence
        self.parameters = kwargs
        super().__init__(*chain([QuestionMarker, QuestionList], extras))
        
    def render_question_marker(self, token):
        assert self.record_answers, "Probably a misplaced question marker has been used (i.e., a list not starting with it)"
        if token.marker != ' ':
            self.answers[-1].append(True)
        else:
            self.answers[-1].append(False)            
        return token.question
        
    def render_table_row(self, token):
        cells = [self.render(child) for child in token.children]
        return ' & '.join(cells) + ' \\\\\n'
    
    def render_heading(self, token):
        if token.level > 2:
            return super().render_heading(self, token)
        if token.level == 1:
            return ''
        template = "\question\n{inner}"
        return template.format(inner=self.render_inner(token))
    
    def render_question_list(self, token):
        template = " \\omrchoices{{{choiceno}}}\n\\begin{{choices}}\n{inner}\n\\end{{choices}}\n"
        self.record_answers = True
        self.answers.append([])
        # TODO: get a random permutation, the same for both the mutliple choices and the answers
        answers = ['\n\\choice {inner}'.format(inner=self.render_list_item(child)) for child in token.children]        
        assert len(answers) == len(self.answers[-1])
        if 'shuffle' in self.parameters and self.parameters['shuffle']:
            permutation = list(range(len(answers)))
            random.shuffle(permutation)
            answers = [answers[i] for i in permutation]
            self.answers[-1] = [self.answers[-1][i] for i in permutation]
        self.record_answers = False
        inner = ''.join(answers)
        return template.format(inner=inner, choiceno=len(self.answers[-1]))        
    
    def render_list_item(self, token):
        if not self.record_answers:
            return super().render_list_item(token)
        else:
            return "".join(self.render(child) for child in token.children)
        #    raise Error("Once a question list is started all the list items must be questions")                       
        
    @staticmethod
    def render_thematic_break(token):
        return ''

    def render_document(self, token):
        self.footnotes.update(token.footnotes)
        inner = self.render_inner(token)    
        solutions = []
        for q in self.answers:
            current = ""
            for i in range(len(q)):
                if q[i]:
                    current += chr(ord('A') + i)
            solutions.append(current)
        # encryption of the solution is the default option
        if 'encrypt' not in self.parameters or self.parameters['encrypt']:
            solutions = vigenere_encrypt(','.join(solutions), key=self.parameters['student_no'])
        else:
            solutions = ','.join(solutions)
            
        doc = pylatex.Document(documentclass='omrexam', inputenc = None, lmodern = False, fontenc = None, textcomp = None)
        doc.preamble.append(pylatex.Package('polyglossia'))
        doc.preamble.append(pylatex.Command('setdefaultlanguage', self.parameters['language']))
        doc.preamble.append(pylatex.Command('examname', self.parameters['exam']))
        doc.preamble.append(pylatex.Command('student', arguments=[self.parameters['student_no'], self.parameters['student_name']]))
        doc.preamble.append(pylatex.Command('date', self.parameters['date']))
        doc.preamble.append(pylatex.Command('solution', solutions))
        doc.append(pylatex.NoEscape(self.parameters['header']))        
        with doc.create(QuestionsEnvironment()) as env:
            env.append(pylatex.NoEscape(inner))
        return doc
        
#        return template.format(inner=inner, 
#                               solutions=solutions, 
#                               **self.parameters)
    
class DocumentStripRenderer(LaTeXRenderer):
    def render_table_row(self, token):
        cells = [self.render(child) for child in token.children]
        return ' & '.join(cells) + ' \\\\\n'
    
    def render_document(self, token):
        return self.render_inner(token)
    

In [51]:
def load_questions(directory, config='configuration.md'):
    with open(os.path.join(directory, config), 'r') as f:
        cfg_re = re.compile(r'^\s*---(.*)---\s*(.*)', re.MULTILINE | re.DOTALL)
        m = re.match(cfg_re, f.read())
        configuration = yaml.load(m.group(1))
        content = m.group(2)
    question_re = re.compile(r"\n---")
    title_re = re.compile(r"#\s+.*")
    configuration['files'] = {}
    if 'sources' in configuration:
        for source in configuration['sources']:
            if '*' not in source['file']:
                configuration['files'][source['file']] = {}
                with open(os.path.join(directory, source['file']), 'r') as f:
                    configuration['files'][source['file']]['content'] = list(filter(lambda q: not title_re.match(q), question_re.split(f.read())))
                configuration['files'][source['file']]['select'] = source['select']
            else:
                filenames = set(glob.glob(os.path.join(directory, source['file'])))
                filenames.remove(os.path.join(directory, config))
                for filename in filenames:
                    name = os.path.join(*filename.split(os.path.sep)[1:])
                    if name in configuration['files']:
                        continue
                    configuration['files'][name] = {}
                    with open(filename, 'r') as f:
                        configuration['files'][name]['content'] = list(filter(lambda q: not title_re.match(q), question_re.split(f.read())))
                    configuration['files'][name]['select'] = source['select']                
    configuration['header'] = content
        
    return configuration

In [52]:
def create_exam(configuration, student_no, student_name, date):
    if 'shuffle' not in configuration:
        configuration['shuffle'] = { 'questions': False, 'answers': True }
    # randomly select a given number of questions from each file
    questions = []
    for topic in configuration['files'].values():
        questions += random.sample(topic['content'], topic['select'])
    if 'questions' in configuration['shuffle'] and configuration['shuffle']:
        random.shuffle(questions)
    if 'max_questions' in configuration:
        questions = questions[:configuration['max_questions']]
    if 'header' in configuration:
        with DocumentStripRenderer() as renderer:
            header = renderer.render(Document(configuration['header']))
    else:
        header = ''
    with QuestionRenderer(language=configuration['language'], date=date, 
                          exam=configuration['exam'], student_no=student_no,
                          student_name=student_name, header=header) as renderer:
        content = '\n'.join(questions)
        document = renderer.render(Document(content))
        return document
        

In [53]:
configuration = load_questions('questions')
exam = create_exam(configuration, "34601", "Luca Di Gaspero", "21/02/2019")

In [54]:
import multiprocessing as mp
import pandas as pd
from PyPDF2 import PdfFileReader
from functools import partial
import math
import warnings

In [56]:
def process(digits, page_limit, date, attempts, row):
    index = row[0]
    student = tuple(row[1])
    done = False
    for _ in range(attempts):
        doc = create_exam(configuration, *student, date)
        f = '{{:0{}d}}-{{}}'.format(digits)
        filename = os.path.join('tmp', f.format(index, student[0]))
        doc.generate_pdf(filepath=filename, compiler='latexmk', compiler_args=['-xelatex'])
        # get rid of the xdv file
        os.remove("{}.xdv".format(filename))
        # check the generated output in terms of pages 
        # TODO: it should be done also in terms of the qrcode, number of questions, coherence of answers
        with open("{}.pdf".format(filename), 'rb') as f:
            pdf_file = PdfFileReader(f)
            if pdf_file.getNumPages() <= page_limit:
                done = True
                break 
    if not done:
        warnings.warn("Couldn't get an exam with at most {} pages".format(page_limit))
    

students = pd.read_excel('ListaStudentiEsameExportExcel-5.xls', skiprows=20)
students['Nominativo'] = students['Nome'] + ' ' + students['Cognome']
pool = mp.Pool()
students.sort_values('Cognome', inplace=True)
students.reset_index(inplace=True)
digits = math.ceil(math.log10(students.shape[0]))
pool.map(partial(process, digits, 2, "21/02/2019", 5), students[['Matricola', 'Nominativo']].iterrows())
pool.close()
pool.join()    